## Hypothesis

The company's financial report factors have a causal impact on quarterly returns, with adjustments made to account for the confounding influences of the specified year, quarter, sector, and additional treatment-specific confounders.

In [1]:
import json
import numpy as np
import pandas as pd
from dowhy import CausalModel
from fundamental_feature_engineer import build_fundamental_features, scale_features
import warnings

warnings.filterwarnings('ignore')

In [2]:
balance_sheet = pd.read_csv('./DOW30_balance_sheet.csv')
cash_flow = pd.read_csv('./DOW30_cash_flow.csv')
income_statement = pd.read_csv('./DOW30_income_statement.csv')
stock_prices = pd.read_csv('./DOW30.csv')

balance_sheet['date'] = pd.to_datetime(balance_sheet['date'])
cash_flow['date'] = pd.to_datetime(cash_flow['date'])
income_statement['date'] = pd.to_datetime(income_statement['date'])
stock_prices['date'] = pd.to_datetime(stock_prices['date'])
stock_prices['return'] = stock_prices.groupby('ticker')['adjClose'].pct_change()

In [3]:
# Pivoting the dataframes
pivot_balance_sheet = balance_sheet.pivot_table(index=['ticker', 'date', 'year', 'quarter'], columns='dataCode', values='value').reset_index()
pivot_cash_flow = cash_flow.pivot_table(index=['ticker', 'date', 'year', 'quarter'], columns='dataCode', values='value').reset_index()
pivot_income_statement = income_statement.pivot_table(index=['ticker', 'date', 'year', 'quarter'], columns='dataCode', values='value').reset_index()

financial_data = pivot_balance_sheet.merge(pivot_cash_flow, on=['ticker', 'date', 'year', 'quarter'], suffixes=('_balance', '_cash'))
financial_data = financial_data.merge(pivot_income_statement, on=['ticker', 'date', 'year', 'quarter'], suffixes=('', '_income'))
financial_data = financial_data.loc[~(financial_data.quarter == 0)] # remove annual one

In [4]:
financial_data = build_fundamental_features(financial_data)
financial_data.tail()

dataCode,ticker,date,year,quarter,accoci,acctPay,acctRec,assetsCurrent,assetsNonCurrent,cashAndEq,...,return_on_capital_employed,long_term_debt_to_capitalization,receivables_turnover,fixed_asset_turnover,dividend_payout_ratio,dividend_coverage_ratio,cash_ratio,operating_margin,cash_flow_margin,equity_multiplier
440,WMT,2023-04-30,2023,1,-1.114700e+10,5.426800e+10,7.647000e+09,7.851100e+10,1.665420e+11,1.057500e+10,...,0.013188,0.492685,19.916438,0.914490,-0.811181,-1.232770,1.358731,0.040971,0.030420,3.384476
441,WMT,2023-07-31,2023,2,-1.081800e+10,5.657600e+10,7.891000e+09,8.203200e+10,1.730890e+11,1.388800e+10,...,0.045640,0.470086,20.483082,0.933809,-0.190488,-5.249674,1.451505,0.045263,0.083944,3.206810
442,WMT,2023-10-31,2023,3,-1.157300e+10,6.104900e+10,8.625000e+09,8.839100e+10,1.707830e+11,1.215400e+10,...,0.005760,0.466083,18.643942,0.941569,-2.385692,-0.419166,0.815158,0.038569,0.005056,3.261856
444,WMT,2024-01-31,2023,4,-1.130200e+10,5.681200e+10,8.796000e+09,7.687700e+10,1.755220e+11,9.867000e+09,...,0.032657,0.453660,19.712142,0.987842,-0.270166,-3.701434,1.509408,0.041837,0.096385,3.009730
445,WMT,2024-04-30,2024,1,-1.136700e+10,5.607100e+10,9.075000e+09,7.715200e+10,1.769020e+11,9.405000e+09,...,0.030875,0.462255,17.797025,0.912980,-0.314867,-3.175943,0.974813,0.042357,0.026308,3.125165


In [5]:
# Function to find the closest trading day
def find_closest_next_trading_day(target_date):
    future_dates = stock_prices[stock_prices['date'] >= target_date]
    if len(future_dates):
        return future_dates.iloc[0]['date']
    return None

def find_closest_previous_trading_day(target_date):
    past_dates = stock_prices[stock_prices['date'] < target_date] # can't be exactly at the end of report day
    if len(past_dates):
        return past_dates.iloc[-1]['date']
    return None

def calculate_next_returns(df):
    df = df.sort_values('date')
    df['start_date'] = df['date'].apply(find_closest_next_trading_day)
    df['next_date'] = df['date'].shift(-1).apply(find_closest_previous_trading_day)
    df = df.merge(stock_prices[['ticker', 'date', 'adjClose']].rename(columns={'date': 'start_date'}), on=['ticker', 'start_date'], how='left')
    df = df.merge(stock_prices[['ticker', 'date', 'adjClose']].rename(columns={'date': 'next_date', 'adjClose': 'next_adjClose'}), on=['ticker', 'next_date'], how='left')
    df['quarterly_return'] = (df['next_adjClose'] - df['adjClose']) / df['adjClose']
    return df


unique_dates = financial_data[['ticker', 'date']].drop_duplicates().sort_values(['ticker', 'date'])
returns_data = unique_dates.groupby('ticker').apply(calculate_next_returns).reset_index(drop=True)
returns_data = returns_data.dropna()
financial_data = financial_data.dropna(axis=1) # the lastest report will be dropped as it has no next report (next_date)

In [6]:
fundamental_data = pd.merge(financial_data, returns_data[['ticker', 'date', 'quarterly_return']], on=['ticker', 'date'])

In [7]:
# choose a subset to study, where the IC > 0.02 and the confounders abs corr > 0.4 with the feature
# this may lead to a group of similar indicators (multicollinearity) in the end, 
# but as the rest confounders are controlled, it increases the robustness

numeric_corr = fundamental_data.select_dtypes(include=np.number).drop(columns=['year', 'quarter']).corr()
numeric_return_corr = numeric_corr['quarterly_return'].sort_values(key=abs, ascending=False)

feature_study_list = numeric_return_corr[numeric_return_corr.abs() > 0.02].drop('quarterly_return').index.tolist()
strong_corrs = numeric_corr[(numeric_corr.abs() > 0.4) & (numeric_corr < 0.8)] # numeric_corr != 1.0
causal_study = {k: [x for x, y in v.items() if not np.isnan(y)] for k, v in strong_corrs[feature_study_list].to_dict().items()}

In [86]:
# save the causal_study mapping, causal_study # {treatment: [confounders]}
with open('./CausalData/causal_study.json', 'w') as json_file:
    json.dump(causal_study, json_file, indent=4)

In [8]:
# add T-Bill

DOW30_sector = pd.read_csv('./DOW30_sector.csv')
DOW30_sector = pd.Series(DOW30_sector.sector.values, index=DOW30_sector.ticker).to_dict()
fundamental_data['sector'] = fundamental_data['ticker'].map(DOW30_sector)

rf_rates = pd.read_excel('./RiskFree20Yr.xls', usecols=['TcmDate', 'Tcm10yr'])
rf_rates['TcmDate'] = pd.to_datetime(rf_rates['TcmDate'])
rf_rates = rf_rates.rename({'TcmDate':'date', 'Tcm10yr':'T_Bill'}, axis=1)

In [9]:
fundamental_data.replace([np.inf, -np.inf], np.nan, inplace=True)

# Separate the numeric and non-numeric columns
numeric_cols = fundamental_data.select_dtypes(include=np.number).columns
non_numeric_cols = fundamental_data.select_dtypes(exclude=np.number).columns

# Fill missing values in numeric columns with the median
fundamental_data[numeric_cols] = fundamental_data[numeric_cols].fillna(fundamental_data[numeric_cols].median())

# Combine the numeric and non-numeric columns back together
fundamental_data = pd.concat([fundamental_data[numeric_cols], fundamental_data[non_numeric_cols]], axis=1)

# Encode columns using one-hot encoding
data = pd.get_dummies(fundamental_data, columns=['ticker', 'quarter', 'sector']) # don't controll the year

rf_rates.set_index('date', inplace=True)
full_date_range = pd.date_range(start=data.date.min(), end=data.date.max(), freq='D')
rf_rates = rf_rates.reindex(full_date_range).ffill().bfill().reset_index()
rf_rates.rename(columns={'index': 'date'}, inplace=True)

data = data.merge(rf_rates, on='date', how='left')

In [10]:
non_numeric_columns = data.select_dtypes(exclude=np.number).columns
data = scale_features(data, 'quarterly_return', non_numeric_columns)

In [91]:
# save the preprocessed data
data.to_csv('./CausalData/DOW30_fundamental.csv', index=False)

In [ ]:
estimates = []

ticker_list = data.columns[data.columns.str.contains('ticker.*')].tolist()
quarter_list = data.columns[data.columns.str.contains('quarter.*') & ~data.columns.str.contains('quarterly_return')].tolist()
sector_list = data.columns[data.columns.str.contains('sector.*')].tolist()

for treatment_feature, treat_confounders in causal_study.items():
    outcome_feature = 'quarterly_return'
    confounders = ticker_list + quarter_list + treat_confounders + sector_list + ['T_Bill']

    model = CausalModel(
        data=data,
        treatment=treatment_feature,
        outcome=outcome_feature,
        common_causes=confounders
    )
    
    identified_estimand = model.identify_effect()
    causal_estimate = model.estimate_effect(identified_estimand, method_name="backdoor.linear_regression")
    estimates.append((treatment_feature, causal_estimate, model, identified_estimand))
    print(f"Linear Regression Estimate for {treatment_feature}:", causal_estimate.value)



In [15]:
def process_causal_estimate(metric_name, causal_estimate, model, identified_estimand):
    # print(f"Processing {metric_name}...")

    result = {
        'metric_name': metric_name,
        'causal_estimate': causal_estimate.value,
    }

    # Perform refutation tests
    refute_placebo = model.refute_estimate(
        identified_estimand, causal_estimate, method_name="placebo_treatment_refuter"
    )

    refute_subset = model.refute_estimate(
    identified_estimand, causal_estimate, method_name="data_subset_refuter"
    )

    refute_random = model.refute_estimate(
        identified_estimand, causal_estimate, method_name="random_common_cause"
    )

    result['refute_placebo_new_effect'] = refute_placebo.new_effect
    result['refute_placebo_p_value'] = refute_placebo.refutation_result['p_value']
    result['refute_subset_new_effect'] = refute_subset.new_effect
    result['refute_subset_p_value'] = refute_subset.refutation_result['p_value']
    result['refute_random_new_effect'] = refute_random.new_effect
    result['refute_random_p_value'] = refute_random.refutation_result['p_value']

    return result

In [11]:
# save the causal estimates result
import joblib
# joblib.dump(estimates, 'estimates.joblib')
estimates = joblib.load('estimates.joblib')

In [ ]:
# Process the estimates and refutations
report = []
for feat, causal_estimate, model, identified_estimand in estimates:
    if causal_estimate is not None:
        res = process_causal_estimate(feat, causal_estimate, model, identified_estimand)
        report.append(res)
    else:
        print(f"No valid estimate for {feat}")

In [100]:
final_result = pd.DataFrame(report)
final_result = final_result[(final_result.refute_subset_p_value > 0.05) & (final_result.refute_random_p_value > 0.05)]

In [101]:
final_result = final_result.sort_values('causal_estimate', key=abs, ascending=False).reset_index(drop=True)

In [20]:
# The placebo tests suggest that the observed effects are likely genuine,
# as the placebo new effects are almost zero and the p-values are statistically significant.
# The random and subset tests indicate that the causal effects are robust to random perturbations,
# as the new effect estimates remain close to the original, and the p-values are generally not significant.

final_result.sample(5)

,metric_name,causal_estimate,refute_placebo_new_effect,refute_placebo_p_value,refute_subset_new_effect,refute_subset_p_value,refute_random_new_effect,refute_random_p_value
15,consolidatedIncome,0.069470,2.137179e-15,0.0,0.074794,0.90,0.069406,1.00
28,operating_return_on_assets,0.034343,-4.507505e-14,0.0,0.033614,0.94,0.034470,0.96
3,ppeq,0.201731,-2.223569e-14,0.0,0.216249,0.88,0.201470,0.98
48,epsDil,-0.000555,6.938894e-17,0.0,-0.000310,0.94,-0.000628,0.90
34,issrepayEquity,0.022182,1.579639e-14,0.0,0.023679,0.96,0.022248,0.90


In [109]:
final_result.to_csv('./CausalData/study_result.csv', index=False)